GC Net Data

In [2]:
!pip install pyarrow

You should consider upgrading via the '/Users/nilsfulde/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install git+https://github.com/GEUS-PROMICE/pyNEAD.git

  Cloning https://github.com/GEUS-PROMICE/pyNEAD.git to /private/var/folders/_r/2f7t_cz14232jqp3576_p04m0000gn/T/pip-req-build-lq7_zv60
  Running command git clone --filter=blob:none -q https://github.com/GEUS-PROMICE/pyNEAD.git /private/var/folders/_r/2f7t_cz14232jqp3576_p04m0000gn/T/pip-req-build-lq7_zv60
  Resolved https://github.com/GEUS-PROMICE/pyNEAD.git to commit d811220c2d048d84d4612b045880d6ca914c39a4
  Preparing metadata (setup.py) ... done
You should consider upgrading via the '/Users/nilsfulde/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [4]:
#!/usr/bin/env bash

# Latest L1 data: https://github.com/GEUS-Glaciology-and-Climate/GC-Net-level-1-data-processing/tree/main/L1
# API contents of latest L1 data (raw URLs etc.): https://api.github.com/repositories/319306521/contents/L1

import os
os.getcwd()
os.chdir('C:\\Users\\nifu18ab\\Desktop\\GEUS-Master-Thesis')
os.chdir("../")
print(os.getcwd())


try:
    os.mkdir("data")
    os.mkdir("data/data_daily")
    os.mkdir("data/data_hourly")
except:
    print('Overwritting existing data in "/data"')

import urllib.request

# Download data
print("Downloading daily data...\r")


# xargs -n 1 curl --silent -O --output-dir data_daily < ../metadata/urls_1.txt
for url in open("metadata/urls_1.txt"):
    # Split on the rightmost / and take everything on the right side of that
    name = url.rsplit("/", 1)[-1].replace("\r", "")
    # Strip /n at the end of filename
    name = name.strip()
    # Combine the name and the downloads directory to get the local filename
    filename = os.path.join("data/data_daily", name)

    # Download the file if it does not exist
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)


/Users/nilsfulde/Documents/University/University/CPH/Master/2_Semester/Machine_Learning/Exam/FairFace/GEUS-Master-Thesis
Overwritting existing data in "/data"


In [5]:
print("Downloading hourly data...\r")
# xargs -n 1 curl --silent -O --output-dir data_hourly < ../metadata/urls_2.txt
for url in open("metadata/urls_2.txt"):
    # Split on the rightmost / and take everything on the right side of that
    name = url.rsplit("/", 1)[-1].replace("\r", "")
    # Strip /n at the end of filename
    name = name.strip()
    # Combine the name and the downloads directory to get the local filename
    filename = os.path.join("data/data_hourly", name)

    # Download the file if it does not exist
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)

In [6]:
# Process data
# echo -ne 'Processing dairly data...\r'
# python scripts/process_data_daily.py
# echo -ne 'Processing hourly data...\r'
# python scripts/process_data_hourly.py

# Delete unprocessed data
# rm -r data_daily data_hourly

# %% process data daily
import pandas as pd
import nead

# Convert NEAD files to Pandas dataframes
station = pd.read_csv("metadata/station_info.csv", header=0)
dfs_daily = []

for name, ID in zip(station.Name, station.ID):
    format_name = name.replace(" ", "")
    files = "data/data_daily/" + str(ID).zfill(2) + "-" + format_name + "_daily.csv"
    ds_daily = nead.read(files, index_col=0)
    df_daily = ds_daily.to_dataframe()
    df_daily.insert(
        loc=0, column="station_name", value=name
    )  # Add station_name column to each dataframe
    dfs_daily.append(df_daily)

# Concatenate dataframes
df_daily = pd.concat(dfs_daily).sort_index()

# Delete irrelevant columns from dataframe (i.e. null columns and flag columns)
# null_columns = df_daily.columns[df_daily.isnull().all()]
# flag_columns = df_daily.filter(regex="flag$").columns
# print(null_columns)
# print(flag_columns)

df_daily = df_daily.drop(
    columns=[
        "OSWR_max",
        "HW2_adj_flag",
        "P_adj_flag",
        "HW1_adj_flag",
        "OSWR_adj_flag",
        "HS1_adj_flag",
        "HS2_adj_flag",
        "TA3_adj_flag",
        "TA4_adj_flag",
        "DW1_adj_flag",
    ]
)
# Add season column to dataframe
seasons = {
    1: "Winter",
    2: "Winter",
    3: "Spring",
    4: "Spring",
    5: "Spring",
    6: "Summer",
    7: "Summer",
    8: "Summer",
    9: "Autumn",
    10: "Autumn",
    11: "Autumn",
    12: "Winter",
}

# Extract the month from the index and use the dictionary to map it to the corresponding season
df_daily["season"] = df_daily.index.month.map(seasons)

# Add year column to dataframe
df_daily["year"] = df_daily.index.strftime("%Y")

# Add month column to dataframe
df_daily["month"] = df_daily.index.strftime("%B")

# Rename Index Column to Datetime
df_daily = df_daily.reset_index(inplace=False)
df_daily = df_daily.rename(columns={'timestamp': 'Datetime'}, inplace=False) 

# Rename 'station_name' to file
df_daily = df_daily.rename(columns={'station_name': 'file'}, inplace=False)

# Add Day of Year & Day of Century 
df_daily['DayOfYear'] = df_daily['Datetime'].dt.dayofyear 
df_daily['DayOfCentury'] = df_daily['Datetime'].dt.dayofyear+365*(df_daily['Datetime'].dt.year-1)


# # Add day column to dataframe
# df_daily["day"] = df_daily.index.strftime("%d")

# Add hour column to dataframe
# df_daily["hour"] = df_daily.index.strftime("%h")

# Change column headers
#header = pd.read_csv('/content/drive/MyDrive/Master_Thesis/metadata/Masterdata_GCNET.csv', sep = ";")
header = pd.read_csv('metadata/Masterdata_GCNET.csv', sep = ";")
df_daily = df_daily.rename(columns = header.set_index('fields')['display_description'])

# Save dataframe as parquet file
df_daily.to_parquet("data/df_daily.gzip", compression="gzip", engine='pyarrow')

# %% process data hourly
# Convert NEAD files to Pandas dataframes
station = pd.read_csv("metadata/station_info.csv", header=0)
dfs_hourly = []

for name, ID in zip(station.Name, station.ID):
    format_name = name.replace(" ", "")
    files = "data/data_hourly/" + str(ID).zfill(2) + "-" + format_name + ".csv"
    ds_hourly = nead.read(files, index_col=0)
    df_hourly = ds_hourly.to_dataframe()
    df_hourly.insert(
        loc=0, column="station_name", value=name
    )  # Add station_name column to each dataframe
    dfs_hourly.append(df_hourly)

# Concatenate dataframes
df_hourly = pd.concat(dfs_hourly).sort_index()

# Delete irrelevant columns from dataframe (i.e. null columns and flag columns)
# null_columns = df_hourly.columns[df_hourly.isnull().all()]
# flag_columns = df_hourly.filter(regex="flag$").columns
# print(null_columns)
# print(flag_columns)

df_hourly = df_hourly.drop(
    columns=[
        "OSWR_max",
        "HW2_adj_flag",
        "P_adj_flag",
        "HW1_adj_flag",
        "OSWR_adj_flag",
        "HS1_adj_flag",
        "HS2_adj_flag",
        "TA3_adj_flag",
        "TA4_adj_flag",
        "DW1_adj_flag",
    ]
)
# Add year column to dataframe
df_hourly["year"] = df_hourly.index.strftime("%Y")

# Add month column to dataframe
df_hourly["month"] = df_hourly.index.strftime("%B")

# Add season column to dataframe
seasons = {
    1: "Winter",
    2: "Winter",
    3: "Spring",
    4: "Spring",
    5: "Spring",
    6: "Summer",
    7: "Summer",
    8: "Summer",
    9: "Autumn",
    10: "Autumn",
    11: "Autumn",
    12: "Winter",
}

# Extract the month from the index and use the dictionary to map it to the corresponding season
df_hourly["season"] = df_hourly.index.month.map(seasons)


#Rename Index Column to Datetime
df_hourly = df_hourly.reset_index(inplace=False)
df_hourly = df_hourly.rename(columns={'timestamp': 'Datetime'}, inplace=False) 

# Rename 'station_name' to file
df_hourly = df_hourly.rename(columns={'station_name': 'file'}, inplace=False)

# Add Day of Year & Day of Century 
df_hourly['DayOfYear'] = df_hourly['Datetime'].dt.dayofyear 
df_hourly['DayOfCentury'] = df_hourly['Datetime'].dt.dayofyear+365*(df_hourly['Datetime'].dt.year-1)


# # Add day column to dataframe
# df_hourly["day"] = df_hourly.index.strftime("%d")

# # Add hour column to dataframe
# df_hourly["hour"] = df_hourly.index.strftime("%h")


#header = pd.read_csv('/content/drive/MyDrive/Master_Thesis/metadata/Masterdata_GCNET.csv', sep = ";")
header = pd.read_csv('metadata/Masterdata_GCNET.csv', sep = ";")
df_hourly = df_hourly.rename(columns = header.set_index('fields')['display_description'])

# Save dataframe as parquet file
df_hourly.to_parquet("data/df_hourly.gzip", compression="gzip", engine='pyarrow')

New Promice

In [ ]:
import pandas as pd
import numpy as np
import os

def load_csv_files(csv_directory="..\..\PROMICE-AWS-toolbox\out\L4"):
    
    # List all CSV files in the directory
    csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]

    # Combine all CSV files into a single DataFrame
    dfs = []
    for f in csv_files:
        df = pd.read_csv(os.path.join(csv_directory, f), index_col=False)
        df.insert(0, 'stid', f[:-7])
        dfs.append(df)
    df = pd.concat(dfs)

    #read metadata from promice repository
    station = pd.read_csv('..\..\PROMICE-AWS-toolbox\metadata\AWS_station_locations.csv', index_col=False)
    station.to_csv('..\\data\\new_promice\\AWS_station_locations.csv', index=False)

    output_file = "..\\Data\\new_promice\\all_promice_data_hourly.parquet.gzip"
    df.to_parquet(output_file, compression='gzip', engine='pyarrow')

    #display(output station and columns summary)
    print('Stations loaded:')
    display(df['stid'].unique())
    print('columns in dataset:')
    print(list(df.columns))
    print("FINISHED LOADING CSV's")
    return df


def process_hourly_data(dataframe, directory="../data/new_promice/all_promice_data.parquet.gzip", add_meta_data=False):

    if not isinstance(dataframe, pd.DataFrame):
        df_hourly = pd.read_parquet(directory)
    else:
        df_hourly = dataframe.copy()

    # Add year column to dataframe
    df_hourly["Datetime"] = pd.to_datetime(df_hourly.time)

    #Rename Index Column to Datetime
    df_hourly = df_hourly.reset_index(inplace=False)

    if add_meta_data==True:
        add_nice_to_haves(df_hourly, 'hourly')

    #display(df_hourly.head(10))

    # Convert the DataFrame to a compressed Parquet file
    output_file = "../data/new_promice/all_promice_data_hourly.parquet.gzip"
    df_hourly.to_parquet(output_file, compression='gzip', engine='pyarrow')

    print("FINISHED PROCESSING HOURLY DATA")
    return df_hourly

def process_daily_data(dataframe=None, directory="", add_meta_data=False):
    
    if not isinstance(dataframe, pd.DataFrame):
        dataframe = pd.read_parquet(directory)
    else:
        df = dataframe.copy()

    # Define the date column that you want to group by (replace "date_column" with the name of your column)
    min_values_per_day = 20

    # Group the data by weather station and date
    grouped = df.groupby(['stid','date'])

    # Specify columns containing numerical values to be averaged
    columns_to_average = (df
                .select_dtypes(exclude=['object'])
                .drop(columns=['index', 'Datetime', 'DayOfYear', 'DayOfCentury'])
                .columns
    )
    # Calculate the number of non-NaN values for each variable within each group
    valid_date_observations = grouped[columns_to_average].apply(lambda x: x.notna().sum() <= min_values_per_day)

    # Calculate average per day per station
    df_filtered = grouped[columns_to_average].mean()

    #Remove means with less than 20 observations per day
    df_masked = df_filtered.mask(valid_date_observations, np.nan)

    df_daily = df_masked.reset_index().copy()

    df_daily["Datetime"] = pd.to_datetime(df_daily['date'])

    if add_meta_data==True:
        df_daily = add_nice_to_haves(df_daily, 'daily')

    # Convert the DataFrame to a compressed Parquet file
    output_file = "..\\Data\\new_promice\\all_promice_data_daily.parquet.gzip"
    df_daily.to_parquet(output_file, compression='gzip', engine='pyarrow')

    print("FINISHED PROCESSING DAILY DATA")
    # display(df_daily)
    return df_daily

def process_monthly_data(dataframe=None, directory="", add_meta_data=False):

    if not isinstance(dataframe, pd.DataFrame):
        df = pd.read_parquet(directory)
    else:
        df = dataframe.copy()

    df['month_year'] = df['Datetime'].dt.to_period('M')
    # Define the date column that you want to group by (replace "date_column" with the name of your column)
    min_values_per_month = 24

    # Create a new column with the month and year of the date column

    # Group the data by weather station and date
    grouped = df.groupby(['stid','month_year'])

    # Specify columns containing numerical values to be averaged
    columns_to_average = (df
                .select_dtypes(exclude=['object'])
                .drop(columns=['Datetime', 'DayOfYear', 'DayOfCentury'])
                .columns
    )

    # Calculate the number of non-NaN values for each variable within each group
    valid_date_observations = grouped[columns_to_average].apply(lambda x: x.notna().sum() <= min_values_per_month)

    # Calculate average per day per station
    df_filtered = grouped[columns_to_average].mean()

    #Remove means with less than 20 observations per day
    df_masked = df_filtered.mask(valid_date_observations, np.nan)

    df_monthly = df_masked.reset_index().copy()

    df_monthly["Datetime"] = pd.to_datetime(df_monthly['month_year'].astype(str) + '-01')

    if add_meta_data==True:
        df_monthly = add_nice_to_haves(df_monthly, 'monthly')

    #display(df_monthly)

    # Convert the DataFrame to a compressed Parquet file
    output_file = "..\\Data\\new_promice\\all_promice_data_monthly.parquet.gzip"
    df_monthly.to_parquet(output_file, compression='gzip', engine='pyarrow')

    print('FINISHED PROCESSING MONTHLY DATA')
    return df_monthly

###########################################################################################
### Helper functions

def add_station_info(df):
    station = pd.read_csv('../data/new_promice/AWS_station_locations.csv')
    station.rename(columns={'timestamp':'station_location_timestamp'})
    df.merge(station, how='left',on=['stid','stid'])
    return df

# helper functions for adding metadata like season, day, month, year, date, DayOfYear and DayOfCentury
def add_nice_to_haves(df, period):
    def add_season(df):
        seasons = {
            1: "Winter",
            2: "Winter",
            3: "Spring",
            4: "Spring",
            5: "Spring",
            6: "Summer",
            7: "Summer",
            8: "Summer",
            9: "Autumn",
            10: "Autumn",
            11: "Autumn",
            12: "Winter",
        }

        # Extract the month from the index and use the dictionary to map it to the corresponding season
        df["season"] = df['Datetime'].dt.month.map(seasons)
        return df

    def add_common(df):
        df["year"] = df['Datetime'].dt.strftime("%Y")
        df["month"] = df['Datetime'].dt.strftime("%B")
        df["date"] = df['Datetime'].dt.date
        df['DayOfYear'] = df['Datetime'].dt.dayofyear 
        df['DayOfCentury'] = df['Datetime'].dt.dayofyear+365*(df['Datetime'].dt.year-1)
        df = add_season(df)
        return df
        
    def add_day(df):
        # Add day column to dataframe
        df["day"] = df['Datetime'].dt.strftime("%d")
        return df

    def add_hour(df):
        # Add hour column to dataframe
        df["hour"] = df['Datetime'].dt.strftime("%h")
        return df

    if period == 'hourly':
        df = add_hour(
            add_day(
            add_common(df)))

    elif period == 'daily':
        df = add_day(
            add_common(df))
    elif period == 'monthly':
        df = add_common(df)
    
    return df 

#### Unused code
    # Process data
    # echo -ne 'Processing dairly data...\r'
    # python scripts/process_data_daily.py
    # echo -ne 'Processing hourly data...\r'
    # python scripts/process_data_hourly.py

    # Delete unprocessed data
    # rm -r data_daily data_hourly

    # %% process data hourly
    # Reading station metadata

    # Delete irrelevant columns from dataframe (i.e. null columns and flag columns)
    # null_columns = df_hourly.columns[df_hourly.isnull().all()]
    # flag_columns = df_hourly.filter(regex="flag$").columns
    # print(null_columns)
    # print(flag_columns)

    #df_hourly = df_hourly.drop(
    #    columns=[
    #           'gps_geounit', 
    #           'batt_v_ini', 
    #           'freq_vw',
    #    ]
    #)

In [ ]:
df = load_csv_files()

header = pd.read_csv('metadata/promice_header.csv', sep = ";")

df_hourly = process_hourly_data(dataframe=df, add_metadata=True)
df_hourly = df_hourly.rename(columns = header.set_index('standard_name')['long_name'])

df_hourly = process_hourly_data(directory="../data/new_promice/all_promice_data.parquet.gzip")
############################################

df_daily = process_daily_data(dataframe=df_hourly, add_meta_data=True)
df_daily = df_daily.rename(columns = header.set_index('standard_name')['long_name'])

df_daily = process_daily_data(directory="../data/new_promice/all_promice_data_hourly.parquet.gzip", add_meta_data=True)

############################################
df_monthly = process_monthly_data(dataframe=df_daily, add_meta_data=True)
df_monthly = df_monthly.rename(columns = header.set_index('standard_name')['long_name'])

df_monthly = process_monthly_data(directory='..\\Data\\new_promice\\all_promice_data_daily.parquet.gzip', add_meta_data=True)

### helper function to add station information such as # of booms, location and classification

# df_hourly = add_station_info(df_hourly)
# df_daily = add_station_info(df_daily)
# df_monthly = add_station_info(df_monthly)

In [ ]:
null_columns = df_hourly.columns[df_hourly.isnull().all()]
flag_columns = df_hourly.filter(regex="flag$").columns
print("null columns:\n", null_columns)
print("flag_columns:\n", flag_columns)

In [ ]:
df_monthly.info

Promice Data

In [7]:
!wget -r -e robots=off -nH --cut-dirs=3 --content-disposition "https://dataverse.geus.dk/api/datasets/:persistentId/dirindex?persistentId=doi:10.22008/FK2/8SS7EW"

--2023-02-16 12:34:59--  https://dataverse.geus.dk/api/datasets/:persistentId/dirindex?persistentId=doi:10.22008/FK2/8SS7EW
Resolving dataverse.geus.dk (dataverse.geus.dk)... 194.182.159.137
Connecting to dataverse.geus.dk (dataverse.geus.dk)|194.182.159.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15287 (15K) [text/html]
Saving to: ‘.index.html’

.index.html         100%[===================>]  14,93K  --.-KB/s    in 0,02s   

2023-02-16 12:34:59 (904 KB/s) - ‘.index.html’ saved [15287/15287]

--2023-02-16 12:34:59--  https://dataverse.geus.dk/api/access/datafile/34687
Reusing existing connection to dataverse.geus.dk:443.
HTTP request sent, awaiting response... 200 OK
Length: 545593 (533K) [text/plain]
Saving to: ‘CEN_day_v03.txt’

CEN_day_v03.txt     100%[===================>] 532,81K  --.-KB/s    in 0,07s   

2023-02-16 12:35:00 (7,70 MB/s) - ‘CEN_day_v03.txt’ saved [545593/545593]

--2023-02-16 12:35:00--  https://dataverse.geus.dk/api/access/datafil

HTTP request sent, awaiting response... 200 OK
Length: 42578809 (41M) [text/plain]
Saving to: ‘KPC_L_hour_v03.txt’

KPC_L_hour_v03.txt  100%[===================>]  40,61M  18,0MB/s    in 2,3s    

2023-02-16 12:35:09 (18,0 MB/s) - ‘KPC_L_hour_v03.txt’ saved [42578809/42578809]

--2023-02-16 12:35:09--  https://dataverse.geus.dk/api/access/datafile/34658
Reusing existing connection to dataverse.geus.dk:443.
HTTP request sent, awaiting response... 200 OK
Length: 31315 (31K) [text/plain]
Saving to: ‘KPC_L_month_v03.txt’

KPC_L_month_v03.txt 100%[===================>]  30,58K  --.-KB/s    in 0s      

2023-02-16 12:35:09 (61,8 MB/s) - ‘KPC_L_month_v03.txt’ saved [31315/31315]

--2023-02-16 12:35:09--  https://dataverse.geus.dk/api/access/datafile/34691
Reusing existing connection to dataverse.geus.dk:443.
HTTP request sent, awaiting response... 200 OK
Length: 1740498 (1,7M) [text/plain]
Saving to: ‘KPC_U_day_v03.txt’

KPC_U_day_v03.txt   100%[===================>]   1,66M  --.-KB/s    in 0

HTTP request sent, awaiting response... 200 OK
Length: 102563 (100K) [application/pdf]
Saving to: ‘PROMICE_coordinates.pdf’

PROMICE_coordinates 100%[===================>] 100,16K  --.-KB/s    in 0,02s   

2023-02-16 12:35:16 (5,35 MB/s) - ‘PROMICE_coordinates.pdf’ saved [102563/102563]

--2023-02-16 12:35:16--  https://dataverse.geus.dk/api/access/datafile/34683
Reusing existing connection to dataverse.geus.dk:443.
HTTP request sent, awaiting response... 200 OK
Length: 382615 (374K) [text/plain]
Saving to: ‘QAS_A_day_v03.txt’

QAS_A_day_v03.txt   100%[===================>] 373,65K  --.-KB/s    in 0,007s  

2023-02-16 12:35:16 (53,7 MB/s) - ‘QAS_A_day_v03.txt’ saved [382615/382615]

--2023-02-16 12:35:16--  https://dataverse.geus.dk/api/access/datafile/34719
Reusing existing connection to dataverse.geus.dk:443.
HTTP request sent, awaiting response... 200 OK
Length: 9346490 (8,9M) [text/plain]
Saving to: ‘QAS_A_hour_v03.txt’

QAS_A_hour_v03.txt  100%[===================>]   8,91M  40,0M

TAS_A_day_v03.txt   100%[===================>]   1,03M  --.-KB/s    in 0,03s   

2023-02-16 12:35:23 (35,9 MB/s) - ‘TAS_A_day_v03.txt’ saved [1081094/1081094]

--2023-02-16 12:35:23--  https://dataverse.geus.dk/api/access/datafile/34700
Reusing existing connection to dataverse.geus.dk:443.
HTTP request sent, awaiting response... 200 OK
Length: 26439289 (25M) [text/plain]
Saving to: ‘TAS_A_hour_v03.txt’

TAS_A_hour_v03.txt  100%[===================>]  25,21M  35,8MB/s    in 0,7s    

2023-02-16 12:35:24 (35,8 MB/s) - ‘TAS_A_hour_v03.txt’ saved [26439289/26439289]

--2023-02-16 12:35:24--  https://dataverse.geus.dk/api/access/datafile/34656
Reusing existing connection to dataverse.geus.dk:443.
HTTP request sent, awaiting response... 200 OK
Length: 19786 (19K) [text/plain]
Saving to: ‘TAS_A_month_v03.txt’

TAS_A_month_v03.txt 100%[===================>]  19,32K  --.-KB/s    in 0,001s  

2023-02-16 12:35:24 (23,6 MB/s) - ‘TAS_A_month_v03.txt’ saved [19786/19786]

--2023-02-16 12:35:24--  ht

HTTP request sent, awaiting response... 200 OK
Length: 1600357 (1,5M) [text/plain]
Saving to: ‘UPE_U_day_v03.txt’

UPE_U_day_v03.txt   100%[===================>]   1,53M  --.-KB/s    in 0,03s   

2023-02-16 12:35:30 (44,9 MB/s) - ‘UPE_U_day_v03.txt’ saved [1600357/1600357]

--2023-02-16 12:35:30--  https://dataverse.geus.dk/api/access/datafile/34708
Reusing existing connection to dataverse.geus.dk:443.
HTTP request sent, awaiting response... 200 OK
Length: 39148729 (37M) [text/plain]
Saving to: ‘UPE_U_hour_v03.txt’

UPE_U_hour_v03.txt  100%[===================>]  37,33M  41,3MB/s    in 0,9s    

2023-02-16 12:35:31 (41,3 MB/s) - ‘UPE_U_hour_v03.txt’ saved [39148729/39148729]

--2023-02-16 12:35:31--  https://dataverse.geus.dk/api/access/datafile/34648
Reusing existing connection to dataverse.geus.dk:443.
HTTP request sent, awaiting response... 200 OK
Length: 28858 (28K) [text/plain]
Saving to: ‘UPE_U_month_v03.txt’

UPE_U_month_v03.txt 100%[===================>]  28,18K  --.-KB/s    in

In [9]:
# save hourly data

from pathlib import Path
import pandas as pd
import numpy as np

#path = r'/content'  # or unix / linux / mac path
path = r'data/promice_data'

# Get the files from the path provided in the OP
files = Path(path).glob('*_hour_v03.txt')  # .rglob to get subdirectories

dfs = list()
for f in files:
    data = pd.read_csv(f, delimiter=r"\s+", engine='python')
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

df = df.replace(-999, np.nan)

df["Month"] = df["MonthOfYear"]
df["Day"] = df["DayOfMonth"]
df["Hour"] = df["HourOfDay(UTC)"]

df["Datetime"] = pd.to_datetime(df[["Year", "Month", "Day", "Hour"]], format='%Y%m%d%h')

# Save dataframe as parquet file
df.to_parquet("data/promice_hourly.gzip", compression="gzip", engine='pyarrow')
#df.to_parquet("/content/drive/MyDrive/Master_Thesis/data/promice_hourly.gzip", compression="gzip")

In [10]:
# save daily data 
# Get the files from the path provided in the OP
files = Path(path).glob('*_day_v03.txt')  # .rglob to get subdirectories

dfs = list()
for f in files:
    data = pd.read_csv(f, delimiter=r"\s+", engine='python')
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

df = df.replace(-999, np.nan)

df["Month"] = df["MonthOfYear"]
df["Day"] = df["DayOfMonth"]

df["Datetime"] = pd.to_datetime(df[["Year", "Month", "Day"]], format='%Y%m%d')

# Save dataframe as parquet file
df.to_parquet("data/promice_daily.gzip", compression="gzip", engine='pyarrow')
#df.to_parquet("/content/drive/MyDrive/Master_Thesis/data/promice_daily.gzip", compression="gzip")

In [11]:
# save monthly data 
# Get the files from the path provided in the OP
files = Path(path).glob('*_month_v03.txt')  # .rglob to get subdirectories

dfs = list()
for f in files:
    data = pd.read_csv(f, delimiter=r"\s+", engine='python')
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

df = df.replace(-999, np.nan)

df["Month"] = df["MonthOfYear"]

df["Datetime"] = pd.to_datetime(['{}-{}-01'.format(y, m) for y, m in zip(df.Year, df.Month)])

# Save dataframe as parquet file
df.to_parquet("data/promice_monthly.gzip", compression="gzip", engine='pyarrow')
#df.to_parquet("/content/drive/MyDrive/Master_Thesis/data/promice_monthly.gzip", compression="gzip")